In [ ]:
import numpy as np
import pandas as pd
import regex as re
import math
import time

mentees = pd.read_csv('../DATA/Mentee-Application-Form.csv')
mentors = pd.read_csv('../DATA/Mentor-Application-Form.csv')

In [ ]:
def JaccardSim(listA, listB):
    if len(listA) == 0 or len(listB) == 0:
        return 0.0
    return len(set(listA).intersection(listB)) / len(set(listA).union(listB))

In [ ]:
def MenteeSimilarityMentor(allmentees, allmentors, t, tupleBinary):
    start = time.time()
    
    similarities = []
    for i in range(len(allmentees)):

        similarity = []
        menteeInterests = str(allmentees.iloc[i, 3]).split(';')
        menteeClubs = str(allmentees.iloc[i, 4]).split(';')

        for j in range(len(allmentors)):

            weight = allmentors.iloc[j, 5]
            if math.isnan(weight):
                weight = 1
            
  
            mentorInterests = str(allmentors.iloc[j, 8]).split(';')
            mentorClubs = str(allmentors.iloc[j, 9]).split(';')

            jaccardInterest = JaccardSim(menteeInterests, mentorInterests)
            jaccardClubs = JaccardSim(menteeClubs, mentorClubs)

            similarity.append([weight * jaccardInterest, weight * jaccardClubs, j])
            
        if t == 'ic':
            similarities.append(list(reversed(sorted([[x[0] + x[1], x[2]] for x in similarity], key = lambda x : x[0]))))
        elif t == 'i':
            similarities.append(list(reversed(sorted([[x[0], x[2]] for x in similarity], key = lambda x : x[0]))))
        elif t == 'c':
            similarities.append(list(reversed(sorted([[x[1], x[2]] for x in similarity], key = lambda x : x[0]))))
        else:
            raise ValueError('Wrong tuning parameter')

    sims = []
    for i in range(len(similarities)):
        sims.append(list(map(lambda x : x[-1], similarities[i])))

    if tupleBinary == 0:
        names = []
        for i in range(len(similarities)):
            names.append(list(map(lambda x : allmentors.iloc[x[-1], 1], similarities[i])))
            
    elif tupleBinary == 1:
        names = []
        for i in range(len(similarities)):
            names.append(list(map(lambda x : allmentors.iloc[x[-1], 1] + '#' +  str(int(x[0])), similarities[i])))
        
    else:
        raise ValueError('Please Enter a tuple parameter of 1 or 0')

    names = pd.DataFrame(names, index= list(allmentees.iloc[:,1])).transpose()
    
    end = time.time()
    
    return names

In [ ]:
def match2(studentpreferences, allcoaches, t, MenteeNumConstraint):
    
    def getmaxtuple(lstoflsts):
        res = lstoflsts[0]
        for i in range(1, len(lstoflsts)):
            if lstoflsts[i][1] > res[1]:
                res = lstoflsts[i]
        return res
    
    names = [x.split('@')[0] for x in list(allcoaches.iloc[:, 1])]
    if MenteeNumConstraint == 0:
        d = dict((x, 0) for x in names)
    elif MenteeNumConstraint == 1:
        yesNo = list(map(lambda x : t if x == 'Yes' else 1, list(allcoaches.iloc[:,4])))
        d = dict()
        for i in range(len(yesNo)):
            if yesNo[i] == 1:
                d[names[i]] = 0
            else:
                d[names[i]] = t - 1
    else:
        raise ValueError('last parameter must be 0 or 1')
    
    matching = []
    paired = []
    
    for j in range(len(studentpreferences.index)):

        mentors = [[x.split('#')[0], int(x.split('#')[1])] for x in list(studentpreferences.iloc[j, :])]
        
        while getmaxtuple(mentors)[1] > 0:

            maxtuple = getmaxtuple(mentors)
            mentor = maxtuple[0]
            ind = mentors.index(maxtuple)
            student = studentpreferences.columns[ind]
            
            if d[mentor] < t and not(student in paired):
                d[mentor] += 1
                matching.append([mentor, student])
                paired.append(student)
            
            mentors[ind][1] = 0

    matching = pd.DataFrame(matching, columns=['mentor', 'mentee'])
    return matching